# Unsloth Environment Verification

This notebook verifies that all components are correctly installed for running Unsloth notebooks:
- GRPO (Reinforcement Learning)
- Vision fine-tuning (Ministral VL)
- fast_inference support

**Run this after rebuilding the jupyter pod to verify the environment.**

**Note:** This notebook does NOT use vLLM fast_inference. For fast_inference tests, see:
- `01_FastInference_Llama.ipynb` - Llama-3.2-1B
- `02_FastInference_Qwen.ipynb` - Qwen3-4B
- `03_FastInference_Ministral.ipynb` - Ministral models
- `04_Vision_Training.ipynb` - Vision training pipeline

In [1]:
# Load environment variables from .env file
from dotenv import load_dotenv
import os

# Load .env from notebook directory
load_dotenv()
print(f"✓ HF_TOKEN loaded: {'Yes' if os.environ.get('HF_TOKEN') else 'No'}")

# CRITICAL: Import unsloth FIRST for proper TRL patching
import unsloth
from unsloth import FastLanguageModel, FastVisionModel
print(f"✓ unsloth: {unsloth.__version__}")

import transformers
print(f"✓ transformers: {transformers.__version__}")

import vllm
print(f"✓ vLLM: {vllm.__version__}")

import trl
print(f"✓ TRL: {trl.__version__}")

import torch
print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")

✓ HF_TOKEN loaded: Yes🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.

/opt/pixi/.pixi/envs/default/lib/python3.13/site-packages/trl/__init__.py:203: UserWarning: TRL currently supports vLLM versions: 0.10.2, 0.11.0, 0.11.1, 0.11.2. You have version 0.14.0rc1.dev201+gadcf682fc.cu130 installed. We recommend installing a supported version to avoid compatibility issues.
  if is_vllm_available():

🦥 Unsloth Zoo will now patch everything to make training faster!✓ unsloth: 2025.12.10
✓ transformers: 5.0.0rc1
✓ vLLM: 0.14.0rc1.dev201+gadcf682fc
✓ TRL: 0.26.2
✓ PyTorch: 2.9.1+cu130
✓ CUDA available: True
✓ GPU: NVIDIA GeForce RTX 4080 SUPER

In [2]:
# Test imports for Reinforcement Learning notebook
print("=== GRPO/RL Imports ===")
from trl import GRPOConfig, GRPOTrainer
from datasets import Dataset
print("✓ All GRPO imports successful")

=== GRPO/RL Imports ===
✓ All GRPO imports successful

In [3]:
# Test imports for Vision notebook
print("=== Vision/SFT Imports ===")
from trl import SFTTrainer, SFTConfig
from unsloth.trainer import UnslothVisionDataCollator
from unsloth import is_bf16_supported
from transformers import TextStreamer
print("✓ All Vision imports successful")

=== Vision/SFT Imports ===
✓ All Vision imports successful

In [4]:
# Test model loading with FastLanguageModel (standard inference, no vLLM)
print("=== Testing Model Loading ===")
model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Ministral-3-3B-Reasoning-2512",
    max_seq_length=2048,
    load_in_4bit=True,
)
print(f"✓ Model loaded: {type(model).__name__}")
print(f"✓ Tokenizer: {type(tokenizer).__name__}")
print("✓ FastLanguageModel test PASSED")
del model, tokenizer

=== Testing Model Loading =====((====))==  Unsloth 2025.12.10: Fast Ministral3 patching. Transformers: 5.0.0rc1. vLLM: 0.14.0rc1.dev201+gadcf682fc.cu130.
   \\   /|    NVIDIA GeForce RTX 4080 SUPER. Num GPUs = 1. Max memory: 15.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu130. CUDA: 8.9. CUDA Toolkit: 13.0. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

Loading weights:   0%|          | 0/458 [00:00<?, ?it/s]

✓ Model loaded: Mistral3ForConditionalGeneration
✓ Tokenizer: PixtralProcessor
✓ FastLanguageModel test PASSED

In [5]:
# Test fast_inference capability
print("=== Fast Inference Check ===")
print("fast_inference=True uses vLLM as backend for 2x faster inference")
print(f"vLLM version: {vllm.__version__}")
print(f"Unsloth version: {unsloth.__version__}")

# Check if fast_inference is supported
import inspect
sig = inspect.signature(FastLanguageModel.from_pretrained)
if 'fast_inference' in sig.parameters:
    print("✓ fast_inference parameter available")
else:
    print("⚠ fast_inference parameter not found")

# Check FastVisionModel
sig_vision = inspect.signature(FastVisionModel.from_pretrained)
has_fast_inference_vision = 'fast_inference' in sig_vision.parameters
print(f"✓ fast_inference in FastVisionModel: {has_fast_inference_vision}")

=== Fast Inference Check ===
fast_inference=True uses vLLM as backend for 2x faster inference
vLLM version: 0.14.0rc1.dev201+gadcf682fc
Unsloth version: 2025.12.10
✓ fast_inference parameter available
✓ fast_inference in FastVisionModel: True

In [6]:
# Shutdown kernel to release all GPU memory
import IPython
print("Shutting down kernel to release GPU memory...")
app = IPython.Application.instance()
app.kernel.do_shutdown(restart=False)

Shutting down kernel to release GPU memory...

{'status': 'ok', 'restart': False}

## Verification Summary

If all cells above ran without errors, your environment is ready for:

1. **Reinforcement Learning** (GRPO/TRL imports verified)
2. **Vision Fine-tuning** (SFT imports verified)
3. **fast_inference** (parameter available)

### Next Steps

Run the specialized fast_inference notebooks (each requires fresh kernel):
- `01_FastInference_Llama.ipynb` - Llama-3.2-1B with vLLM
- `02_FastInference_Qwen.ipynb` - Qwen3-4B with vLLM
- `03_FastInference_Ministral.ipynb` - Ministral models
- `04_Vision_Training.ipynb` - Complete vision training pipeline